In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

22/05/18 20:06:57 WARN Utils: Your hostname, CarlPC resolves to a loopback address: 127.0.1.1; using 172.27.169.216 instead (on interface eth0)
22/05/18 20:06:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/18 20:06:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
lines = sc.textFile("/home/nolfonzo/src/Essential-PySpark-for-Scalable-Data-Analytics/README.md")
words = lines.flatMap(lambda s: s.split(" "))
word_tuples = words.map(lambda s: (s, 1))
word_count = word_tuples.reduceByKey(lambda x, y: x + y)
word_count.take(10)

[('#', 1),
 ('PySpark', 7),
 ('Scalable', 3),
 ('Analytics', 1),
 ('', 53),
 ('src="https://static.packt-cdn.com/products/9781800568877/cover/smaller"',
  1),
 ('Analytics"', 1),
 ('height="256px"', 1),
 ('align="right"></a>', 1),
 ('is', 10)]

In [4]:
from pyspark.sql.functions import split, explode
from pyspark.sql import SparkSession
spark = SparkSession(sc)
linesDF = spark.read.text("/home/nolfonzo/src/Essential-PySpark-for-Scalable-Data-Analytics/README.md")

In [5]:
linesDF.show()

+--------------------+
|               value|
+--------------------+
|# Essential PySpa...|
|                    |
|<a href="https://...|
|                    |
|This is the code ...|
|                    |
|**A beginner's gu...|
|                    |
|## What is this b...|
|Apache Spark is a...|
|                    |
|This book covers ...|
|Understand the ro...|
|Gain an appreciat...|
|Scale out your da...|
|Build data pipeli...|
|Leverage the clou...|
|Explore the appli...|
|Integrate your cl...|
|                    |
+--------------------+
only showing top 20 rows



In [6]:
wordListDf = linesDF.select(split("value", " ").alias("words"))
wordListDf.show()
wordsDf = wordListDf.select(explode("words").alias("word"))
wordsDf.show()

+--------------------+
|               words|
+--------------------+
|[#, Essential, Py...|
|                  []|
|[<a, href="https:...|
|                  []|
|[This, is, the, c...|
|                  []|
|[**A, beginner's,...|
|                  []|
|[##, What, is, th...|
|[Apache, Spark, i...|
|                  []|
|[This, book, cove...|
|[Understand, the,...|
|[Gain, an, apprec...|
|[Scale, out, your...|
|[Build, data, pip...|
|[Leverage, the, c...|
|[Explore, the, ap...|
|[Integrate, your,...|
|                  []|
+--------------------+
only showing top 20 rows

+--------------------+
|                word|
+--------------------+
|                   #|
|           Essential|
|             PySpark|
|                 for|
|            Scalable|
|                Data|
|           Analytics|
|                    |
|                  <a|
|href="https://www...|
|src="https://stat...|
|      alt="Essential|
|             PySpark|
|                 for|
|            Scalable|
|       

In [7]:
wordCountDf = wordsDf.groupBy("word").count()
wordCountDf.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|              lakes,|    1|
|[copy](https://ww...|    1|
|              today!|    1|
|          Dashboards|    1|
|                 API|    1|
|                  If|    1|
| .add('Description',|    1|
|                used|    1|
|         **Following|    1|
|           analysts,|    1|
|          practicing|    1|
|                 PDF|    1|
|                Data|    5|
|             present|    1|
|                   =|    1|
|             helping|    1|
|          beginner's|    1|
|               go-to|    1|
|         Databricks,|    1|
|          border="5"|    1|
+--------------------+-----+
only showing top 20 rows



In [9]:
spark.sql("drop table if exists word_counts")
spark.sql('CREATE TABLE word_counts (word STRING) USING csv OPTIONS("delimiter"=" ") LOCATION "/home/nolfonzo/src/Essential-PySpark-for-Scalable-Data-Analytics/README.md"')

spark.sql("SELECT word, COUNT(word) AS count FROM word_counts GROUP BY word").show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                  If|    1|
|         **Following|    1|
|                   *|    2|
|alt="https://www....|    1|
|                null|    0|
|            Leverage|    1|
|              Apache|    1|
|                  <a|    2|
|                  is|    1|
|                 All|    1|
|                   ||    3|
|                Gain|    1|
|           **Sreeram|    1|
|               Scale|    1|
|           Integrate|    1|
|                   )|    1|
|             Explore|    1|
|                With|    1|
|                 The|    1|
|                 ```|    2|
+--------------------+-----+
only showing top 20 rows

